In [1]:
import tensorflow as tf
from tensorflow import keras

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"   

In [2]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, epsilon=0.001):
        super().__init__()
        self.epsilon = epsilon
        
    def build(self, input_shape):
        self.alpha = self.add_weight(
            shape=input_shape[-1:],
            initializer=keras.initializers.Ones(),
            trainable=True,
            dtype=tf.float32
        )
        self.betta = self.add_weight(
            shape=input_shape[-1:], 
            initializer=keras.initializers.Zeros(), 
            trainable=True,
            dtype=tf.float32
        )
        
    def call(self, input_tensor):
        mean, variance = tf.nn.moments(input_tensor, axes=-1, keepdims=True)
        std = tf.math.sqrt(variance)
        return self.alpha * (X - mean)/(std + self.epsilon) + self.betta
    
    def get_config(self):
        return {"epsilon": self.epsilon}

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [4]:
import numpy as np

X = X_train.astype(np.float32)

custom_layer_norm = LayerNormalization()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=2.1054968e-06>

In [5]:
random_alpha = np.random.rand(X.shape[-1])
random_beta = np.random.rand(X.shape[-1])

custom_layer_norm.set_weights([random_alpha, random_beta])
keras_layer_norm.set_weights([random_alpha, random_beta])

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=1.1342989e-06>